# Trabalho 3

### Introdução

Atualmente sabe-se que algoritmos quânticos podem comprometer as técnicas criptográficas convencionais. Assim é necessário recorrer a técnicas criptográficas pós-quânticas que permitam resistir a ataques de computadores quânticos.  
Neste trabalho foi pedida uma implementação KEM-IND-CPA e uma implementação PKE-IND-CCA sobre dois dos algoritmos pós quânticos atualmente viáveis nomeadamente:

- NTRU
- NewHope  
  
  
Explicaremos ambas as implementações pela ordem em que são referidas para cada um dos algoritmos.

### NTRU

#### KEM-IND-CPA

##### KeyGen()

- O primeiro passo é a geração de um elemento g pequeno em R, g tem de ser invertivel em R/3. Existem várias maneiras de garantir isso, nós fizemos uso da função is_unit() sobre R3(g).
- Em seguida geramos um f pequeno em R com peso w. De seguida calculamos a inversa de g á qual chamamos g_inv.
- Por fim guardamos a chave secreta (f , g_inv) e calculamos a chave pública  Rq(g)/Rq(3*f) .  
  
  
##### Encapsulate()

- Começamos por calcular um polinómio r pequeno de peso w em R.
- De seguida calculamos um Hash do r ao qual chamamos key. E calculamos também o criptograma C composto pela multiplicação r*pk em q, sendo pk a chave pública, seguida de um arredondamento para o múltiplo de 3 mais próximo. 
- Retornamos o par (key,C).  
  
  
##### Decapsulate()

- O primeiro passo é a multiplicação de criptograma por 3*f em Rq, guardando o resultado na variavel pre_process.
- Em seguida, pondo cada coeficiente do resultado anterior entre −(q − 1)/2 e (q − 1)/2, reduzimos módulo 3 obtendo "e" em R/3.
- "e" é igual a g*r em R3, então para obtermos r' basta-nos multiplicar "e" pela inversa de g aplicando em seguida o lift. 
- Por fim fizemos o Hash do r' e devolvemos comparando o resultado obtido com a chave devolvida no Encrypt verificando a correta implementação do algoritmo.

#### PKE-IND-CPA
  
  
##### KeyGen()  
  
Esta função manteve-se igual á função impleentada na versão KEM-IND-CPA. 
  
##### Encapsulate()  

- Nesta função começamos por proceder á geração de um polinómio r pequeno de peso w em R. 
- De seguida foi gerado um polinómio salt (s <-{0,1}^t). 
- Calculamos o criptograma como r*pk + Rq(m) como referido na página 6 do documento NTRU-PRIME.pdf "c = m + hr in R/q, where m is small."
- Por fim calculamos a key fazendo um HMAC sobre o r usando o salt como chave e devolvemos o par (key,c).
  

##### Decapsulate()  
 
- Começamos por abrir a chave secreta em (f, g_inv)
- Á semelhança do Decapsulate feito no esquema KEM-IND-CPA calculamos o r. Posteriormente fizemos o HMAC sobre esta para verificar a correção do algoritmo.
  
### NewHope

#### KEM-IND-CPA  
  
##### KeyGen()  
  
  
##### Encapsulate()  
  
  
##### Decapsulate()  


#### PKE-IND-CPA
  
  
##### KeyGen()  
  
  
##### Encapsulate()  
  
  
##### Decapsulate()  

In [60]:
import hashlib
from random import choice, randint


"""
transpõe os coeficientes de "w" para o intervalo -q//2..+q//2
e arredonda-os ao múltiplo de 3 mais próximo
"""
def f(x):
    return ((x/3).round())*3
    
class NTRU_Prime():
    def __init__(self,w):
        self.w=w #único parâmetro que inicializa a classe
        q=24*self.w
        while True:
            if (1+q).is_prime():
                break 
            else:
                q += 1         
        q=q+1
        
        Zx.<x>  = ZZ[]
        Zq.<z>  = PolynomialRing(GF(q))
    
        p = next_prime(2*self.w)
        while True:
            if  Zq(x^p-x-1).is_irreducible():
                break
            else:
                p = next_prime(p+1)
                
        self.p=p
        self.q=q
    
    
    def small_poly(self,p,t=None):
        """
        polinómios cujos coeficientes são -1, 0, 1
        """
        Zx.<x>  = ZZ[]
        if not t:
            return Zx([choice([-1,0,1]) for k in range(p)])
        u = floor(2*(p-1)//t) ; k = randint(0,u) ; l = [0]*p
        while k < p:
            l[k] = choice([-1,1]) ; k += randint(1,u)
        return Zx(l)
    def Hash(self,t): #função para calcular o hash de um objecto
        ww = reduce(lambda x,y: x + y.binary(), t.list() , "")
        return hashlib.sha256(ww.encode('utf-8')).hexdigest()
    
    
    def round_3(self,t):
        Zx.<x>  = ZZ[]
        r = self.q//2
        pol_list = t.list()
        res = [f(lift(p+r) - r) for p in pol_list]
        return Zx(res) 
    
    
    def round_(self,t,n=-1):
        if n==-1:
            n=self.q
        Zx.<x>  = ZZ[]
        """
        input:  polinómio em Gqr ou Z3r
        output: transpõe os coeficientes para o intervalo -n//2..+n//2
        """
        r = n//2
        res_list = []
        pol_list = t.list()
        for p in pol_list:
            res_list.append(lift(p+r) - r)
        return Zx(res_list)
            

    
    def keygen(self):
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        g = self.small_poly(self.p)
        while not R3(g).is_unit():  # enquanto R3(g) não for invertível, geramos novo g.
            g = self.small_poly(self.p)
        f = self.small_poly(self.p,self.w)
        g_inv = R3(g)^(-1)
        self.secret = (f , g_inv)
        self.pk = Rq(g)/Rq(3*f)  # chave pública

        
    def encapsulate(self):
        #preparação comum
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        #geração de um polinómio pequeno
        r = self.small_poly(self.p,self.w)
        key = self.Hash(r)
        C   = self.round_3(Rq(r)*self.pk)
        return (key, C)
    
    
    def decapsulate(self,C):
        #preparação comum
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        # Decode c, obtaining c ∈ R.
        (f , g_inv) = self.secret
        #Multiply by 3f in R/q.
        #View each coefficient of 3fc in R/q as an integer between −(q − 1)/2 and (q − 1)/2,
        # reduce modulo 3, obtaining a polynomial e in R/3
        pre_process = Rq(3*f) * Rq(C)
        e = g_inv * R3(self.round_(pre_process)) ;
        w = self.round_(e,n=3) ;
        key = self.Hash(w)
        return key

In [61]:
A=NTRU_Prime(286)
A.keygen()
(key,C) = A.encapsulate()
key == A.decapsulate(C)





True

In [1]:
import hashlib, hmac
from sage.crypto.util import ascii_integer
from random import choice, randint


"""
transpõe os coeficientes de "w" para o intervalo -q//2..+q//2
e arredonda-os ao múltiplo de 3 mais próximo
"""
def f(x):
    return ((x/3).round())*3
    
"""
concatena 2 polinómios
"""
def merge(a,b):
    s = []
    if len(a) != len(b):
        print("unequal sizes")
        return False
    for i in range(len(a)):
        s.append(a[i]+b[i])
    return s
    
    
def _toZ(f,p=None):
    ff = list(f)
    if p == None:
        return ff
    else:
        fp = map(lift,[Mod(a,p) for a in ff])
        return [u if u <= p//2 else u-p for u in fp ]
    
'''
Util para converter listas de inteiros para uma string que prepresente esses inteiros por ordem correta
'''
def _toStr(intList):
    bin = BinaryStrings()
    s = [str(i) for i in intList]
    return bin.encoding("".join(s))
    

    
def _toBytes(salt):
    bts = [int(s).to_bytes( 1 , byteorder='big', signed=True ) for s in salt]
    res = b''.join(bts)
    return res
    
class NTRU_Prime():
    def __init__(self,w):
        self.w=w #único parâmetro que inicializa a classe
        q=24*self.w
        while True:
            if (1+q).is_prime():
                break 
            else:
                q += 1         
        q=q+1
        
        Zx.<x>  = ZZ[]
        Zq.<z>  = PolynomialRing(GF(q))
    
        p = next_prime(2*self.w)
        while True:
            if  Zq(x^p-x-1).is_irreducible():
                break
            else:
                p = next_prime(p+1)
                
        self.p=p
        self.q=q
        self.salt = []
    

    def msg_gen(self):
        return  [choice([-1,0,1]) for k in range(self.q)]
    
    def small_poly(self,p,t=None):
        """
        polinómios cujos coeficientes são -1, 0, 1
        """
        Zx.<x>  = ZZ[]
        if not t:
            return Zx([choice([-1,0,1]) for k in range(p)])
        u = floor(2*(p-1)//t) ;
        k = randint(0,u);
        l = [0]*p
        while k < p:
            l[k] = choice([-1,1]) ;
            k += randint(1,u)
        return Zx(l)
    
    
    def round_3(self,t):
        Zx.<x>  = ZZ[]
        r = self.q//2
        pol_list = t.list()
        res = [f(lift(p+r) - r) for p in pol_list]
        return Zx(res) 
    
    
    def round_(self,t,n=-1):
        if n==-1:
            n=self.q
        Zx.<x>  = ZZ[]
        """
        input:  polinómio em Gqr ou Z3r
        output: transpõe os coeficientes para o intervalo -n//2..+n//2
        """
        r = n//2
        res_list = []
        pol_list = t.list()
        for p in pol_list:
            res_list.append(lift(p+r) - r)
        return Zx(res_list)
            

    
    def keygen(self):
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        g = self.small_poly(self.p)
        while not R3(g).is_unit():  # enquanto R3(g) não for invertível, geramos novo g.
            g = self.small_poly(self.p)
        f = self.small_poly(self.p,self.w)
        g_inv = R3(g)^(-1)
        self.secret = (f , g_inv)
        self.pk = Rq(g)/Rq(3*f)  # chave pública

        
    def saltGenerator(self,p):
        """
        polinómio cujos coeficientes são 0, 1
        """
        salt = [choice([0,1]) for k in range(p)]
        
        return salt
    

    def encapsulate(self):
        #preparação comum
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        
        #geração de um polinómio pequeno em R/p
        r = _toZ(self.small_poly(self.p,self.w))
        
        #geração de um salt
        self.salt = self.saltGenerator(len(r))
        btarray = _toBytes(self.salt)
        
        #Criptograma
        C   = self.round_3(Rq(r)*self.pk)
        
        #Key
        key = hmac.new(btarray, _toBytes(r), hashlib.sha1).digest()
        return (key, C)
    
    
    #recebe o objeto C devolvido pela função Encapsulate 
    def decapsulate(self,C):
        #preparação comum
        Zx.<x>  = ZZ[]
        Z3.<y>  = PolynomialRing(GF(3))
        Zq.<z>  = PolynomialRing(GF(self.q))
        R.<x> = Zx.quotient(x^self.p-x-1)
        R3.<y> = Z3.quotient(y^self.p-y-1)
        Rq.<z> = Zq.quotient(z^self.p-z-1)
        
        
        # Abrir a chave secreta
        (f , g_inv) = self.secret
        #1ºpasso
        pre_process = Rq(3*f) * Rq(C)
        #2ºpasso -> obtenção do e 
        e = g_inv * R3(self.round_(pre_process)) ;
        #3ºpasso -> obtenção do r
        w = self.round_(e,n=3);
        wStr = _toStr(w)
        btarray = _toBytes(self.salt)
        key = hmac.new(btarray, _toBytes(w), hashlib.sha1).digest()
        
        return key

In [2]:
A=NTRU_Prime(286)

In [3]:
A=NTRU_Prime(286)
A.p,A.q,A.w

(757, 6869, 286)

In [4]:
A.keygen()

In [5]:
(key,C) = A.encapsulate()

In [6]:
key2 = A.decapsulate(C)
print(key == key2)

True
